In [16]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
import pandas as pd

In [36]:
import random

In [25]:
helph_df = pd.read_csv('heath_news.csv')
medicine = pd.read_csv('medicines.csv')

In [46]:
len(helph_df)

200

In [47]:
helph_df_groupby = helph_df.groupby('publishedDate').first().reset_index()

In [37]:
random.shuffle(medicine['Company'])

In [40]:
medicine_list = medicine['Company'][:100]

In [30]:
Q = """Classify in a range of -1 to 1, the correlation of the following new in the giving text to the availability of the medicine Orloc, where -1 is not correlated and 1 is very correlated. Take consideration the usage of the medicine Orloc and the content of the text. Respond with just  a number.

'\''The rising price of cigarettes is motivating more people - one in four adults in England - to give up smoking, research suggests.\r\nIn a survey of nearly 6,000 people, health concerns were still the t… [+2378 chars]'\''
"""

In [41]:
def get_score(text):
    # To display example params enter
    GenParams().get_example_values()

    generate_params = {
        GenParams.MAX_NEW_TOKENS: 25,
        GenParams.TEMPERATURE: 0.17,
        GenParams.TOP_K: 50,
        GenParams.TOP_P: 1,
        GenParams.REPETITION_PENALTY: 1
    }

    model = Model(
        model_id=ModelTypes.LLAMA_2_70B_CHAT,
        params=generate_params,
        credentials={
            "apikey": "4vQJgBfuBVEvebuIUYza4sLlgBFiAwkpTWUNOJOwmGW5",
            "url": "https://eu-de.ml.cloud.ibm.com"
        },
        project_id="de5b3490-12b1-4e7c-bc07-8b9b517fdd28"
        )

    
    generated_response = model.generate(prompt=text)
    return generated_response['results'][0]['generated_text'].replace('\n', '')

In [49]:
dataBase_medicineScore = pd.DataFrame(columns=['date', 'medicine', 'score'])
for i in range(0, len(helph_df_groupby)):
    news = helph_df_groupby['content'][i]
    date = helph_df_groupby['publishedDate'][i]
    for j in range(0, len(medicine_list)):
        medicine_name = medicine_list[j]
        Q = """Classify in a range of -1 to 1, the correlation of the following new in the giving text to the availability of the medicine {}, where -1 is not correlated and 1 is very correlated. Take consideration the usage of the medicine {} and the content of the text. Respond with just  a number.
        {}
        """.format(medicine_name, medicine_name,news )
        score = get_score(Q)
        # print('The score of the news "{}" with the medicine "{}" is: {}'.format(date, medicine_name, score))
        dataBase_medicineScore = pd.concat([dataBase_medicineScore, pd.DataFrame({'date': [date], 'medicine': [medicine_name], 'score': [score]})]) 
    dataBase_medicineScore.to_csv('output.csv', index=False)
    if i % 50 == 0:
        print('The news {} has been processed'.format(i))
        

The news 0 has been processed


In [32]:
get_score(Q)

'-0.5'